# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
!pip3 install --upgrade pandas

    100% |████████████████████████████████| 10.0MB 3.1MB/s eta 0:00:01   53% |█████████████████▏              | 5.4MB 34.0MB/s eta 0:00:01
    100% |████████████████████████████████| 20.2MB 1.6MB/s eta 0:00:01  3% |█▏                              | 716kB 33.6MB/s eta 0:00:01    31% |██████████                      | 6.4MB 16.7MB/s eta 0:00:01    38% |████████████▏                   | 7.7MB 24.3MB/s eta 0:00:01    90% |████████████████████████████▉   | 18.2MB 30.5MB/s eta 0:00:01    97% |███████████████████████████████ | 19.6MB 24.6MB/s eta 0:00:01
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
  Found existing installation: numpy 1.12.1
    Uninstalling numpy-1.12.1:
      Successfully uninstalled numpy-1.12.1
  Found existing installation: pandas 0.23.3
    Uninstalling pandas-0.23.3:
      Successfully uninstalled pandas-0.23.3


In [2]:
# import libraries
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine

import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

from sklearn.multioutput import MultiOutputClassifier

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score , f1_score,  precision_score, recall_score
from sklearn.model_selection import train_test_split
import pickle

In [4]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# load data from database
engine = create_engine('sqlite:///disaster_database.db')
df = pd.read_sql_table('disaster_messages',engine.connect())

In [4]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df["message"][0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [7]:
df = df[df["related"]<2]

In [8]:
X = df["message"].to_numpy()
y = df.drop(columns=["id",'message',"original","genre"])
label_names = y.columns
y=y.to_numpy()

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    lemmatizer = WordNetLemmatizer() 
    #remove punctuation and uwanted characters
    text=re.sub("r[^a-zA-Z0-9]"," ",text)
    #tokenzie the words
    words= nltk.word_tokenize(text)
    #lower case the words and lemmantize them
    words = [lemmatizer.lemmatize(word.lower()) for word in words]
    #remove stop words
    words= [word for word in words if word not in stopwords.words('english')]
    
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline = Pipeline([
    ('bow',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('classifier',MultiOutputClassifier(KNeighborsClassifier())) 
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
clf = pipeline.fit(X_train,y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
ypred= clf.predict(X_test)

In [17]:
print(classification_report(np.array(y_test,ndmin=2),np.array(ypred,ndmin=2), labels=list(range(36)),target_names= label_names ))

                        precision    recall  f1-score   support

               related       0.84      0.90      0.87      3964
               request       0.66      0.52      0.58       912
                 offer       0.50      0.04      0.07        25
           aid_related       0.64      0.58      0.61      2175
          medical_help       0.53      0.17      0.25       393
      medical_products       0.60      0.20      0.30       265
     search_and_rescue       0.65      0.07      0.13       150
              security       0.00      0.00      0.00        91
              military       0.69      0.16      0.27       164
           child_alone       0.00      0.00      0.00         0
                 water       0.60      0.33      0.42       332
                  food       0.61      0.41      0.49       582
               shelter       0.66      0.29      0.40       477
              clothing       0.60      0.28      0.38        86
                 money       0.58      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
filename = 'knn_model.sav'
pickle.dump(clf, open(filename, 'wb'))
 

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.